# Read Simulation Data
With the following code, we have developed a method for reading and transforming simulation data from JaamSim. The method consists of two main parts. The first is the `get_WIP` function, which takes data from an ExpressionLogger, and calculates the average estimated throughput, actual throughput, and Work-in-Process (WIP). The second method processes the `RunOutputList` data, transforming it into a more readable dataframe, and also printing out the LaTeX code for the table.

In [1]:
import pandas as pd

#Set full display of columns
pd.set_option('display.max_columns', None)

# Set full width of the output
pd.set_option('display.width', 1000)

In [2]:
def get_WIP(file_name):
    # Open the file, skip the first 13 lines
    with open(file_name, 'r') as f:
        lines = f.readlines()[12:]

    # Skip 4 rows, keep 3 rows, skip 4 rows, keep 3 rows, etc.
    lines_time = []
    lines_e = []
    lines_a = []
    lines_wip = []

    i = 0
    for line in lines:
        if i % 8 != 0 and i % 8 != 1 and i % 8 != 2 and i % 8 != 3:
            lines_time.append(line.strip().split("\t")[0])
            lines_e.append(line.strip().split("\t")[1])
            lines_a.append(line.strip().split("\t")[2])
            lines_wip.append(line.strip().split("\t")[3])

        i += 1

    # Lines to df
    df = pd.DataFrame(list(zip(lines_time, lines_e, lines_a, lines_wip)), columns =['Time', 'Estimated', 'Actual', 'WIP'])

    # group by time and mean of values
    df['Actual'] = pd.to_numeric(df['Actual'])
    df['Estimated'] = pd.to_numeric(df['Estimated'])
    df['WIP'] = pd.to_numeric(df['WIP'])
    mean = df.groupby('Time').mean()
    mapper = {
        "1020.0" : "17:00",
        "1080.0" : "18:00",
        "1140.0" : "19:00",
        "1200.0" : "20:00"
    }
    
    mean = mean.rename(index=mapper)

    return mean


In [3]:
def get_CT_and_Queue(file_name):
    measures = ['CT_total', 'SD_total', 'CT_FU', 'SD_FU', 'CT_R', 'SD_R', 'CT_N', 'SD_N', 'Queue_Working', 'SQ_Working', 'Queue_Wating_Room', 'SQ_Wating_Room', 'Queue_frontdesk', 'SQ_frontdesk', 'Queue_total_l', 'SQ_total_l', "CT_Working", "SD_Working", 'Queue_total_c', 'SQ_total_c', 'Queue_FU', 'SQ_FU', 'Queue_R', 'SQ_R', 'Queue_N', 'SQ_N']
    # Read the file
    with open(file_name, "r") as file:
        lines = file.readlines()[1:]

    print("Read the file")
    line = lines[0].split("\t")[1:]

    # Make two and two tuples
    data_dict = {}
    for i in range(0, len(measures) * 2, 2):
        measure = measures[i // 2]
        value, conf = line[i], line[i+1]
        if measure.startswith('SD'):
            # Add SD as a new column to the corresponding CT
            ct_key = measure.replace('SD', 'CT')
            if ct_key in data_dict:
                data_dict[ct_key]['SD'] = value
        elif measure.startswith('SQ'):
            # Add SD as a new column to the corresponding Queue
            queue_key = measure.replace('SQ', 'Queue')
            if queue_key in data_dict:
                data_dict[queue_key]['SD'] = value
        else:
            # Create new entry for CT measures or others
            data_dict[measure] = {'Mean': value, 'SD': None, 'Confidence': conf}  # Initialize SD and Confidence as None

    # Create DataFrame
    df = pd.DataFrame.from_dict(data_dict, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Measure'}, inplace=True)

    # Convert columns to numeric and calculate confidence intervals
    df['Mean'] = pd.to_numeric(df['Mean'])
    df['SD'] = pd.to_numeric(df['SD'])
    df['Confidence'] = pd.to_numeric(df['Confidence'])
    df['Confidence_interval_1'] = df['Mean'] - df['Confidence']
    df['Confidence_interval_2'] = df['Mean'] + df['Confidence']
    df  = df.round(3)

    # Create a new column 'Confidence Interval' by concatenating the interval bounds
    df['Confidence Interval'] = "[" + df['Confidence_interval_1'].astype(str) + ", " + df['Confidence_interval_2'].astype(str) + "]"

    #Drop confidence interval columns
    df.drop(columns=['Confidence_interval_1', 'Confidence_interval_2'], inplace=True)
    df.drop(columns=['Confidence'], inplace=True)

    df_cycletime = df.loc[df['Measure'].str.contains('CT')].copy()
    df_cycletime.reset_index(drop=True, inplace=True)

    # Drop last row
    df_cycletime.drop(df_cycletime.tail(1).index, inplace=True)

    # Drop ct prefix
    df_cycletime['Measure'] = df_cycletime['Measure'].str.replace('CT_', '')

    mapper = {
        'total': 'Total',
        'FU': 'Follow-up',
        'R': 'Return',
        'N': 'New'
    }

    df_cycletime['Measure'] = df_cycletime['Measure'].replace(mapper)

    df_queue = df.loc[df['Measure'].str.contains('Queue')].copy()
    df_queue.reset_index(drop=True, inplace=True)

    # Remove Queue prefix
    df_queue['Measure'] = df_queue['Measure'].str.replace('Queue_', '')

    mapper = {
        'Working': 'Waiting for resources',
        'Waiting_Room': 'Waiting Room',
        'frontdesk': 'Front Desk',
        'total_c': 'Total',
        'FU': 'Follow-up',
        'R': 'Return',
        'N': 'New'
    }

    df_queue['Measure'] = df_queue['Measure'].replace(mapper)

    df_queue['Measure'] = df_queue['Measure'].str.replace('total_l', 'Total')
    # remove row 4 /index 3
    df_queue = df_queue.drop(df_queue.index[3])

    return df_cycletime, df_queue



In [4]:
def convert_to_latex(dataframe):
    return dataframe.to_latex(index=False, escape=False)

In [5]:
file_name_dat = "Opening Hours.dat"
file_name_wip = "Opening Hours-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat)
df_wip = get_WIP(file_name_wip)

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())


Read the file
Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &    Mean &     SD & Confidence Interval \\
\midrule
    Total &  89.772 & 33.325 &    [89.114, 90.429] \\
Follow-up &  73.731 & 27.917 &     [72.98, 74.482] \\
   Return &  91.809 & 31.092 &    [90.882, 92.737] \\
      New & 102.892 & 31.804 &  [101.988, 103.796] \\
\bottomrule
\end{tabular}

Queue

\begin{tabular}{lrrl}
\toprule
              Measure &   Mean &     SD & Confidence Interval \\
\midrule
Waiting for resources &  9.224 & 11.112 &      [9.024, 9.424] \\
          Wating_Room & 29.185 & 23.456 &    [28.783, 29.587] \\
           Front Desk &  1.548 &  4.155 &      [1.483, 1.612] \\
                Total & 38.409 & 26.815 &    [37.867, 38.952] \\
            Follow-up & 35.488 & 24.179 &    [34.897, 36.078] \\
               Return & 41.401 & 27.202 &    [40.618, 42.184] \\
                  New & 37.022 & 26.051 &    [36.295, 37.749] \\
\bottomrule
\end{tabular}

WIP

\begin{tabular}{lrrr}
\toprule
{} &  E